**Lifecycle Emissions Assessment Framework for Steel and Transportation**

This code is designed to estimate the environmental impacts of steel production and transportation. It takes input data about steel weight and transport distance, applies emission factors with uncertainties, and models the supply chain processes involved. The code calculates the total emissions, runs simulations to understand uncertainty in the results, and performs sensitivity analysis to see how changes in factors affect the outcomes. 

In [13]:
import random
from typing import Dict, List
import numpy as np

class DataIngestor:
    def __init__(self):
        # Initialize raw data container
        self.raw_data = None
        
    def ingest_data(self, product_data: Dict) -> Dict:
        # Validate and store input product data
        self.raw_data = product_data
        required_fields = ['steel_kg', 'transport_km']
        for field in required_fields:
            if field not in product_data:
                raise ValueError(f"Missing required field: {field}")
        return self.raw_data
    
    def extract_data_from_text(self, text: str) -> Dict:
        # Extract numeric data for steel and transport from text
        data = {}
        lines = text.lower().split('\n')
        for line in lines:
            if 'steel' in line:
                numbers = [float(s) for s in line.split() if s.replace('.', '', 1).isdigit()]
                if numbers:
                    data['steel_kg'] = numbers[0]
            elif 'transport' in line:
                numbers = [float(s) for s in line.split() if s.replace('.', '', 1).isdigit()]
                if numbers:
                    data['transport_km'] = numbers[0]
        if 'steel_kg' not in data or 'transport_km' not in data:
            raise ValueError("Text insufficient to extract all required data.")
        self.raw_data = data
        return data

class EmissionFactorMapper:
    def __init__(self):
        # Initialize base and custom emission factors
        self.base_factors = {
            'steel': {'mean': 2.1, 'uncertainty': 0.2},     
            'transport': {'mean': 0.15, 'uncertainty': 0.1}  
        }
        self.custom_factors = {}
    
    def get_factor(self, material: str) -> Dict:
        # Retrieve emission factor for given material
        if material in self.custom_factors:
            return self.custom_factors[material]
        elif material in self.base_factors:
            return self.base_factors[material]
        else:
            raise ValueError(f"Unknown material: {material}")
    
    def add_custom_factor(self, material: str, mean: float, uncertainty: float):
        # Add or update a custom emission factor
        self.custom_factors[material] = {'mean': mean, 'uncertainty': uncertainty}
    
    def predict_factors(self, material: str, context: Dict) -> Dict:
        # Adjust factors based on context
        base = self.get_factor(material)
        if context.get('region') == 'low_emission':
            adjusted_mean = base['mean'] * 0.8
        else:
            adjusted_mean = base['mean']
        return {'mean': adjusted_mean, 'uncertainty': base['uncertainty']}

class SupplyChainModel:
    def __init__(self, data_ingestor: DataIngestor):
        # Initialize supply chain with ingested data
        self.data = data_ingestor.raw_data
        self.processes = []
    
    def identify_processes(self) -> List[str]:
        # Define relevant supply chain processes
        self.processes = ['steel_production', 'transportation']
        return self.processes
    
    def calculate_process_inputs(self) -> Dict:
        # Return input amounts for each process
        return {
            'steel_production': {'amount': self.data['steel_kg'], 'unit': 'kg'},
            'transportation': {'amount': self.data['transport_km'], 'unit': 'km'}
        }
    
    def model_supply_network(self, graph_data: Dict):
        # Store supply network graph and validate suppliers list
        self.supply_network = graph_data
        for process, suppliers in graph_data.items():
            if not isinstance(suppliers, list):
                raise ValueError(f"Suppliers for {process} should be a list")

class ImpactCalculator:
    def __init__(self, factor_mapper: EmissionFactorMapper, supply_chain: SupplyChainModel):
        # Initialize with factor mapper and supply chain model
        self.factor_mapper = factor_mapper
        self.supply_chain = supply_chain
    
    def calculate_impacts(self, use_mean: bool = True) -> Dict:
        # Calculate impacts using either mean or sampled factors
        process_inputs = self.supply_chain.calculate_process_inputs()
        impacts = {}
        
        steel_factor = self.factor_mapper.get_factor('steel')
        steel_amount = process_inputs['steel_production']['amount']
        if use_mean:
            steel_impact = steel_amount * steel_factor['mean']
        else:
            steel_impact = steel_amount * random.uniform(
                steel_factor['mean'] * (1 - steel_factor['uncertainty']),
                steel_factor['mean'] * (1 + steel_factor['uncertainty'])
            )
        
        transport_factor = self.factor_mapper.get_factor('transport')
        transport_amount = process_inputs['transportation']['amount']
        if use_mean:
            transport_impact = transport_amount * transport_factor['mean']
        else:
            transport_impact = transport_amount * random.uniform(
                transport_factor['mean'] * (1 - transport_factor['uncertainty']),
                transport_factor['mean'] * (1 + transport_factor['uncertainty'])
            )
        
        impacts['steel_production'] = steel_impact
        impacts['transportation'] = transport_impact
        impacts['total'] = steel_impact + transport_impact
        
        return impacts
    
    def predict_impact_scenarios(self, parameters: Dict):
        # Predict impacts with parameter adjustments
        impacts = {}
        base_inputs = self.supply_chain.calculate_process_inputs()
        
        for process, input_data in base_inputs.items():
            material = process.split('_')[0]
            factor = self.factor_mapper.get_factor(material)
            amount = input_data['amount']
            if process in parameters:
                amount *= parameters[process]
            impacts[process] = amount * factor['mean']
        
        impacts['total'] = sum(impacts.values())
        return impacts

class UncertaintyAnalyzer:
    def __init__(self, impact_calculator: ImpactCalculator):
        # Initialize with impact calculator
        self.calculator = impact_calculator
    
    def monte_carlo_simulation(self, iterations: int = 1000) -> Dict:
        # Run Monte Carlo to estimate impact distributions
        results = {
            'steel_production': [],
            'transportation': [],
            'total': []
        }
        
        for _ in range(iterations):
            impacts = self.calculator.calculate_impacts(use_mean=False)
            for key in results:
                results[key].append(impacts[key])
        
        stats = {}
        for key, values in results.items():
            stats[key] = {
                'mean': np.mean(values),
                'std_dev': np.std(values),
                'min': np.min(values),
                'max': np.max(values),
                'median': np.median(values),
                '5th_percentile': np.percentile(values, 5),
                '95th_percentile': np.percentile(values, 95)
            }
        
        return stats
    
    def sensitivity_analysis(self):
        # Analyze sensitivity of impacts to factor changes
        base_impacts = self.calculator.calculate_impacts(use_mean=True)
        sensitivities = {}
        materials = ['steel', 'transport']
        
        for mat in materials:
            original_factor = self.calculator.factor_mapper.get_factor(mat)['mean']
            self.calculator.factor_mapper.add_custom_factor(mat, original_factor * 1.01, 0)
            increased_impacts = self.calculator.calculate_impacts(use_mean=True)
            
            sensitivities[mat] = {
                process: (increased_impacts[process] - base_impacts[process]) / base_impacts[process] / 0.01
                for process in ['steel_production', 'transportation', 'total']
            }
            
            self.calculator.factor_mapper.add_custom_factor(mat, original_factor, 0)
        
        return sensitivities

class LCAClient:
    def __init__(self, product_data: Dict):
        # Setup all components and ingest initial data
        self.data_ingestor = DataIngestor()
        self.factor_mapper = EmissionFactorMapper()
        
        self.data_ingestor.ingest_data(product_data)
        self.supply_chain = SupplyChainModel(self.data_ingestor)
        self.impact_calculator = ImpactCalculator(self.factor_mapper, self.supply_chain)
        self.uncertainty_analyzer = UncertaintyAnalyzer(self.impact_calculator)
    
    def run_full_analysis(self):
        # Run full LCA analysis pipeline and return results
        self.supply_chain.identify_processes()
        
        print("Calculating baseline impacts...")
        baseline_impacts = self.impact_calculator.calculate_impacts()
        
        print("Running Monte Carlo uncertainty analysis...")
        mc_stats = self.uncertainty_analyzer.monte_carlo_simulation(iterations=1000)
        
        print("Performing sensitivity analysis...")
        sensitivities = self.uncertainty_analyzer.sensitivity_analysis()
        
        return {
            'baseline_impacts': baseline_impacts,
            'monte_carlo_stats': mc_stats,
            'sensitivity_analysis': sensitivities
        }


**Life Cycle Assessment (LCA) Analysis Execution and Result Summary**

This code runs a full Life Cycle Assessment (LCA) for a product using input data on steel and transport. It calculates deterministic carbon impacts, performs Monte Carlo simulations to estimate uncertainty, and conducts sensitivity analysis to understand how changes in emission factors affect results.

In [15]:
product_data = {'steel_kg': 1000, 'transport_km': 500}

lca_client = LCAClient(product_data)

results = lca_client.run_full_analysis()

print("Deterministic LCA Results:")
for process, impact in results['baseline_impacts'].items():
    print(f"  {process}: {impact:.2f} kg CO2 eq")

print("\nUncertainty Analysis Results (Monte Carlo):")
for process, stats in results['monte_carlo_stats'].items():
    print(f"{process}:")
    print(f"  Mean: {stats['mean']:.2f} ± {stats['std_dev']:.2f} kg CO2 eq")
    print(f"  Range (5th-95th percentile): {stats['5th_percentile']:.2f} to {stats['95th_percentile']:.2f} kg CO2 eq")

print("\nSensitivity Analysis (Elasticities):")
for material, sens in results['sensitivity_analysis'].items():
    print(f"  {material}:")
    for process, elasticity in sens.items():
        print(f"    {process}: {elasticity:.2f}")


Calculating baseline impacts...
Running Monte Carlo uncertainty analysis...
Performing sensitivity analysis...
Deterministic LCA Results:
  steel_production: 2100.00 kg CO2 eq
  transportation: 75.00 kg CO2 eq
  total: 2175.00 kg CO2 eq

Uncertainty Analysis Results (Monte Carlo):
steel_production:
  Mean: 2094.81 ± 238.95 kg CO2 eq
  Range (5th-95th percentile): 1712.48 to 2477.45 kg CO2 eq
transportation:
  Mean: 74.97 ± 4.40 kg CO2 eq
  Range (5th-95th percentile): 68.24 to 81.96 kg CO2 eq
total:
  Mean: 2169.78 ± 238.84 kg CO2 eq
  Range (5th-95th percentile): 1788.32 to 2547.73 kg CO2 eq

Sensitivity Analysis (Elasticities):
  steel:
    steel_production: 1.00
    transportation: 0.00
    total: 0.97
  transport:
    steel_production: 0.00
    transportation: 1.00
    total: 0.03


***Design**

AI can enhance each module of the LCA model as follows: Natural Language Processing (NLP) can be used in the DataIngestor to automatically extract relevant data from reports or websites. In the EmissionFactorMapper, machine learning can refine and predict emission factors based on context. Graph Neural Networks (GNNs) can model complex relationships in the SupplyChainModel, improving accuracy in tracing inputs and outputs. AI algorithms can also improve ImpactCalculator predictions by learning from past assessments. Finally, Monte Carlo simulation enhanced with AI can provide better uncertainty and sensitivity analysis in the UncertaintyAnalyzer.